<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/6.Gestion_de_datos/Gestion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Gestión de datos: Join, Combine, y Reshape](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=37Vujga1JOOK)

>>[Indice Jerárquico](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=g-QfwPOyJ0JO)

>>[Reordenando los diferentes niveles](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=fW0-QPnwM2X2)

>>[Indexing columnas en un Dataframe](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=5nEdvsjbPN9y)

>[Combinando datasets](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=ONyrRGhIQQaE)

>>[Database-Style joins en Dataframes](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=4aHveRwpQkdS)

>>>[Argumentos de la función Merge](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=S3dWfNjVkOfp)

>>[Merge en el índice](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=SnS0Dvcs0pFL)

>>[Concatenando entre ejes](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=DI3tPxyq4z_l)

>>[Reshaping y Pivot](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=VqNU7hfT7B75)

>>>[Pivotar el formato "largo" a "ancho"](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=AKcrY6W3ADtt)



# Gestión de datos: Join, Combine, y Reshape

En muchas aplicaciones, los datos son distrbuidos a través de un diferentes archivos y base de datos, o en un formato que no es fácil de analizar. 
Para ello, utilizaremos herramientas que nos facilitarán el proceso de preparación de los datos. 
Algunas de ellas ya las hemos visto como parte de otros capítulos, por lo que destacaremos las más importantes.

## Indice Jerárquico

Es una herramienta importante de Pandas. Permite tener multiples índices en un mismo eje. Para ponerlo un poco más simple, te permite trabajar con datos de una dimensión superior en una inferiór (Ejemplo, ventas por local y a través del tiempo)

In [0]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

Lo que está viendo es una vista predefinida de una Serie con un MultiIndex como índice. Los espacios en la visualización del índice significan "usar la etiqueta directamente arriba".
Con este tipo de índices puedes realizar lo que se llama partial-index, lo que permite obtener un subset de los datos.

In [0]:
data['b']

In [0]:
data.loc[:, 2] # La selección es posible incluso desde dentro del nivel

Los índices jerárquicos juegan un rol importante en el modelado y agrupamiento de datos. Por ejemplo, podemos reordenar los datos anteriores en un Dataframe usando el método **unstack**

In [0]:
data.unstack()

In [0]:
data.unstack().stack() # Es la función inversa

## Reordenando los diferentes niveles
Con los Dataframe, los ejes pueden tener índice jerárquicos también. Veamos el siguiente ejemplo:

In [85]:
df = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Marty', 'Marty', 'Doc'],
                              ['Lorraine', 'George', 'Delorean']])
df

Marty             Doc
    Lorraine George Delorean
a 1        0      1        2
  2        3      4        5
b 1        6      7        8
  2        9     10       11

In [0]:
df.index.names = ['key1', 'key2'] # Los niveles pueden tener nombres. 
df

In [0]:
df['Marty'] # Podemos seleccionar datos parciales

In [0]:
df.swaplevel('key1','key2') # También es posible reordenar por niveles

Finalmente, podemos aplicar funciones estadísticas en Dataframes o series, como agregación en un eje particular. 
Considerando el ejemplo anterior, podemos realizar una agregación por nivel, ya sea por fila o columna: 

In [0]:
df.sum(level='key2')

## Indexing columnas en un Dataframe
Es muy común que algunas veces desees mover algunos índices de columnas a filas, o viceversa. Para ello podemos utilizar la función **set_index**

In [92]:
df = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
df

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [0]:
df2 = df.set_index(['c', 'd']) # Si agregamos el parámetro drop = false las columnas no se eliminaran del df
df2

# Combinando datasets

Los datos contenidos en los objetos pandas se pueden combinar de varias maneras:

**pandas.merge** conecta filas en DataFrames en función de una o más claves. Esto será familiar para los usuarios de SQL u otras bases de datos relacionales, ya que implementa operaciones de unión de bases de datos.

**pandas.concat** concatena o "apila" objetos juntos a lo largo de un eje.

El método de instancia **combine_first** permite unir datos superpuestos para completar los valores faltantes en un objeto con valores de otro.

## Database-Style joins en Dataframes

Las operaciones de merge o join combinan conjuntos de datos al vincular filas con una o más claves. Estas operaciones son centrales para las bases de datos relacionales. La función merge en pandas es el principal punto de entrada para usar estos algoritmos en sus datos.

In [94]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
print(df1)
print('\n')
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6


  key  data2
0   a      0
1   b      1
2   d      2


Este es un ejemplo de una unión de muchos a uno; los datos en df1 tienen varias filas etiquetadas con a y b, mientras que df2 tiene solo una fila para cada valor en la columna clave. Llamando a fusionar con estos objetos obtenemos:  

In [97]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Puedes observar que no especifiqué en qué columna unir. Si no se especifica esa información, la combinación usa los nombres de columna superpuestos como las claves. Sin embargo, es una buena práctica especificar explícitamente:

In [0]:
pd.merge(df1, df2, on='key')

Como en las sentencias SQL, si los nombres de las columnas son diferentes en cada objeto, se puede especificar de forma separada.

In [100]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
print (df3)
print('\n')
print(df4)

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6


  rkey  data2
0    a      0
1    b      1
2    d      2


In [0]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

Puedes notar que los valores **c** y **d** y los datos asociados faltan en el resultado. Por defecto, merge hace una unión 'interna' (inner join); Las claves en el resultado son la intersección, o el conjunto común que se encuentra en ambas tablas. Otras opciones posibles son **izquierda**, **derecha** y **exterior**. La unión externa toma la unión de las claves, combinando el efecto de aplicar las uniones izquierda y derecha.
A continuación encontrarás un diagrama con de SQL joins con su correspondiente sentencia en ese lenguaje que te será de utilidad para el desarrollo de tus actividades

<img src = "https://i.pinimg.com/564x/42/48/72/424872ac0b25c05e117b521d55616551.jpg">


A continuación se detallan las opciones que se encuentran disponibles en Pandas con el compartamiento asociado

Opción | Comportamiento
-------|-------
**inner**| Utiliza solo la combinación de claves comunes para ambas tablas
**left** | Utiliza solo la combinación de claves encontradas en la tabla declarada a izquierda
**right** | Utiliza solo la combinación de claves encontradas en la tabla declarada a derecha
**outer** | Utiliza solo la combinación de claves observada en ambas tablas juntas 

In [0]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
print(df1)
print('\n')
print(df2)

In [0]:
pd.merge(df1, df2, how='inner')

Para realizar merge con multiples keys, debemos pasar una lista de nombres de columnas

In [0]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})


In [111]:
print (left)
print ('\n')
print(right)

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3


  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [0]:
pd.merge(left, right, on=['key1', 'key2'], how='inner')

###Argumentos de la función **Merge**

A continuación se detallan los argumentos más utilizados con la función **merge** asociados a su descripción.

Argumento | Descripción
---------|------------
left | Dataframe se fusiona en el lado izquierdo
right | Data frame se fusiona en el lado derecho
how | con parámetro 'inner', 'outer', 'left', o 'right'. Default 'inner'
on | La unión se hace en base a nombre de columnas. Deben estar presentes en ambos Dataframes
left_on| Se utilizan las columnas del Dataframe izquierdo como claves
right_on| Análogo al 'left_on'
left_index|Utiliza  el índice de fila en la izquierda como clave del join.
right_index|Análogo al 'left_index'
sort | Ordena datos fusionados lexicográficamente por las claves.
suffixes|Tupla de valores de cadenas a agregar a una coluna en caso de sobreposición. Si por ejemplo tenemos data en ambos dataframes podemos agregar data_x, data_y como sufijos
copy|Si es falso, evita copiar datos en la estructura resultante.
indicator|Agrega una columna especial '_merge' que indica la fuente de cada fila. Valores pueden ser 'left_only', 'right_only', o, 'both'.




## Merge en el índice

En algunos casos, la clave del merge se dará en el índice. En este caso, puedes pasar el parámetro **left_index=true** o **right_index=true** (o ambos) para indicar que el índice será usado como clave de merge. 


In [0]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
              'value': range(6)})

right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [0]:
print (left1)
print ('\n')
print (right1)

In [0]:
pd.merge(left1, right1, left_on='key', right_index=True) # Qué sucede aquí?

In [0]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer') # y aquí?

In [0]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
            index=['a', 'c', 'e'],
            columns=['Devoto', 'Palermo'])

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
            index=['b', 'c', 'd', 'e'],
            columns=['Belgrano', 'Colegiales'])

otro_df= pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
            index=['a', 'c', 'e', 'f'],
            columns=['Villa Urquiza', 'Nuñez'])

In [120]:
left2

,Devoto,Palermo
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [121]:
right2

,Belgrano,Colegiales
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [122]:
otro_df

,Villa Urquiza,Nuñez
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


Para merge simples de índice sobre índice, puede pasar una lista de DataFrames para unirse como alternativa al uso de la función concat más general que se describe en la siguiente sección

In [0]:
left2.join([right2, otro_df])

## Concatenando entre ejes
Otro tipo de operación de combinación de datos se conoce indistintamente como concatenación, enlace o apilamiento. La función concatenada de NumPy puede hacer esto con las matrices NumPy

In [0]:
arreglo = np.arange(12).reshape((3,4))
arreglo

In [0]:
np.concatenate([arreglo, arreglo], axis=1)

En el contexto de objetos pandas como Series y DataFrame, tener ejes etiquetados le permite generalizar aún más la concatenación de matriz

In [0]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [0]:
pd.concat([s1, s2, s3])

Por defecto, concat funciona a lo largo de axis = 0, produciendo otra serie. Si pasa axis = 1, el resultado será un DataFrame (axis = 1 son las columnas)

In [0]:
pd.concat([s1, s2, s3], axis=1)

In [0]:
pd.concat([s1, s2, s3], axis=1, keys=['uno', 'dos', 'tres'])

## Reshaping y Pivot

Existen varias operaciones básicas para reorganizar datos tabulares. Estos se denominan alternativamente operaciones de Reshaping y Pivot.

La indexación jerárquica proporciona una forma consistente de reorganizar los datos en un DataFrame. Hay dos acciones principales:

**Apilar**
Esto "gira" las columnas en los datos a las filas

**Desapilar**
Esto gira de las filas a las columnas
Ilustraré estas operaciones a través de una serie de ejemplos. Considere un pequeño DataFrame con matrices de cadenas como índices de fila y columna:

In [130]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(['Ohio', 'Colorado'], name='Estado'),
                    columns=pd.Index(['Uno', 'Dos', 'Tres'], name='Número'))
data

Número,Uno,Dos,Tres
Estado,,,
Ohio,0,1,2
Colorado,3,4,5


In [131]:
resultado = data.stack()
resultado

Estado    Número
Ohio      Uno       0
          Dos       1
          Tres      2
Colorado  Uno       3
          Dos       4
          Tres      5
dtype: int64

In [134]:
resultado.unstack()

Número,Uno,Dos,Tres
Estado,,,
Ohio,0,1,2
Colorado,3,4,5


Desapilar podría introducir datos faltantes si no se encuentran todos los valores en el nivel en cada uno de los subgrupos:

In [0]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [137]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [138]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


Cuando se desapila en un DataFrame, el nivel desapilado se convierte en el nivel más bajo en el resultado:

In [139]:
df = pd.DataFrame({'left': resultado, 'right': resultado + 5},
                  columns=pd.Index(['left', 'right'], name='Lado'))
df

Lado             left  right
Estado   Número             
Ohio     Uno        0      5
         Dos        1      6
         Tres       2      7
Colorado Uno        3      8
         Dos        4      9
         Tres       5     10

In [140]:
df.unstack('Estado')

Lado   left          right         
Estado Ohio Colorado  Ohio Colorado
Número                             
Uno       0        3     5        8
Dos       1        4     6        9
Tres      2        5     7       10

In [141]:
df.unstack('Estado').stack('Lado')

Estado        Colorado  Ohio
Número Lado                 
Uno    left          3     0
       right         8     5
Dos    left          4     1
       right         9     6
Tres   left          5     2
       right        10     7

### Pivotar el formato "largo" a "ancho"

Una forma común de almacenar múltiples series de tiempo en bases de datos y CSV es en el llamado formato largo o apilado. Carguemos algunos datos de ejemplo y hagamos una pequeña cantidad de disputas de series de tiempo y otra limpieza de datos:

In [142]:
data = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/6.Gestion_de_datos/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [0]:
periodos = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='fecha')

'''
El método PeriodIndex combina las columnas de año y trimestre
para crear un tipo de intervalo de tiempo.

'''

columnas = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columnas)
data.index = periodos.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'valor'})

In [146]:
ldata[:10]

,fecha,item,valor
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


Este es el llamado formato largo para múltiples series de tiempo u otros datos de observación con dos o más claves (aquí, nuestras claves son fecha y elemento). Cada fila de la tabla representa una sola observación.

Los datos se almacenan con frecuencia de esta manera en bases de datos relacionales como MySQL, ya que un esquema fijo (nombres de columna y tipos de datos) permite que el número de valores distintos en la columna del elemento cambie a medida que se agregan datos a la tabla. En el ejemplo anterior, la fecha y el elemento generalmente serían las claves principales (en el lenguaje de la base de datos relacional), ofreciendo integridad relacional y uniones más fáciles. En algunos casos, los datos pueden ser más difíciles de trabajar en este formato; es posible que prefieras tener un DataFrame que contenga una columna por valor de elemento distinto indexado por marcas de tiempo en la columna de fecha. El método pivote de DataFrame realiza exactamente esta transformación:

In [147]:
pivoted = ldata.pivot('fecha', 'item', 'valor')
pivoted

item,infl,realgdp,unemp
fecha,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


Los primeros dos valores son las columnas que se utilizarán respectivamente como el índice de fila y columna, luego, finalmente, una columna de valor opcional para llenar el DataFrame. Suponga que tiene dos columnas de valor que desea remodelar simultáneamente:

In [148]:
ldata['valor2'] = np.random.randn(len(ldata))
ldata[:10]

,fecha,item,valor,valor2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.272121
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.788349
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.825369
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.364031
4,1959-06-30 23:59:59.999999999,infl,2.340,0.766380
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.905111
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-1.526048
7,1959-09-30 23:59:59.999999999,infl,2.740,0.634709
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.839670
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-0.076978


Omitiendo el último argumento, obtienes un DataFrame con columnas jerárquicas

In [0]:
pivoted = ldata.pivot('fecha', 'item')
pivoted[:5]

In [0]:
pivoted['valor'][:5]

Pivot es equivalente a crear un índice jerárquico usando **set_index** es seguido de una llamada para desapilar

In [0]:
unstacked = ldata.set_index(['fecha', 'item']).unstack('item')
unstacked[:7]

### Pivotar el formato "ancho" a "largo"

Una operación inversa para pivotar en DataFrames es **pandas.melt**. En lugar de transformar una columna en muchas en un nuevo DataFrame, combina varias columnas en una, produciendo un DataFrame que es más largo que la entrada.

In [154]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [155]:
union = pd.melt(df, ['key'])
union

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [0]:
'''
Usando pivot, podemos volver a dar forma al diseño original
'''
reshaped = union.pivot('key', 'variable', 'value')
reshaped

Dado que el resultado de pivot crea un índice a partir de la columna utilizada como etiquetas de fila, es posible que queramos usar reset_index para mover los datos nuevamente a una columna

In [0]:
reshaped.reset_index()

In [0]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])